# *Cargamos data y primer procesamiento*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import sqlite3


# Cambiar directorio a carpeta del proyecto
os.chdir('C:/Python/my_projects/dexboost/')
print(os.getcwd())

C:\Python\my_projects\dexboost


In [2]:
def conectar_db(ruta_db: str):
    try:
        conn = sqlite3.connect(ruta_db)
        print("Conexión exitosa a la base de datos.")
        return conn
    except sqlite3.Error as e:
        print(f"Error al conectar con la base de datos: {e}")
        raise

def cerrar_conexion(conn):
    try:
        if conn:
            conn.close()
            print("Conexión cerrada correctamente.")
    except sqlite3.Error as e:
        print(f"Error al cerrar la conexión: {e}")
        raise

def cargar_tabla(conn, tabla):
    try:
        query = f"SELECT * FROM {tabla}"
        df = pd.read_sql(query, conn)
        return df
    except sqlite3.DatabaseError as e:
        print(f"Error al cargar la tabla '{tabla}': {e}")
        raise

# Conexión a la base de datos
conn = conectar_db('data/main.db')

# Cargar tabla 'Analysis'
data = cargar_tabla(conn, 'analysisLiquidityPool')

# Cerrar conexión

cerrar_conexion(conn)

Conexión exitosa a la base de datos.
Conexión cerrada correctamente.


In [3]:
def initial_processing(df):
    """
    Processes initial df to facilitate later analysis:
        -Convert datetime columns into datetime.
        -Calculate adjustedboostamount
        -Calculate TokenAge (mins)
        -Parse StartLiquidity column to extract liquidity in USD
        -Calculate BoostID for later analysis.
    """

    df = df.copy()

    # Format datetime columns
    df['DetectedAt'] = pd.to_datetime(df['DetectedAt'].astype(str).str[:19])

    # Drop non-interesting columns
    df = df.drop(columns=['id', 'Markets', 'Risks'])

    # Convert TokenAge to minutes (currently in miliseconds)
    df['TokenAge'] = df['TokenAge'] / 60000

    # Change dtypes of each column
    column_dtypes = {
        'TokenAge': 'int32',
        'Amount': 'int16',
        'PubKey': 'string',  
        'IsLP': 'bool',
        'IsPump': 'bool',
        'TokenName': 'string',
        'TokenMint': 'string',
        'MarketCap': 'int64',
        'TotalLiquidity': 'int32',
        'TotalLPProviders': 'int16',
        'RugScore': 'int32'
    }
    df = df.astype(column_dtypes)

    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df


processed_data = initial_processing(data)


In [13]:
cols_to_merge = ['TokenName', 'TokenMint', 'DetectedAt', 'TokenAge', 'MarketCap', 'Amount', 'IsPump', 'TotalLiquidity', 'TotalLPProviders', 'RugScore']
data_to_merge = processed_data[cols_to_merge]

data_to_merge

,TokenName,TokenMint,DetectedAt,TokenAge,MarketCap,Amount,IsPump,TotalLiquidity,TotalLPProviders,RugScore
0,GPT-4.5,62C7rbvbSiHeU8s4LXmmirSxrkH3VKJA1En8ndopump,2025-02-26 16:58:33,84,129729,10,True,35462,0,0
1,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,2025-02-26 18:00:03,8,1416408467,10,False,341774,0,10500
2,he's called pablo,6FgVrXLXZTdRvG3oJepcneM52FqN9iVYUrUsHucrpump,2025-02-26 18:08:53,15,15420,10,True,12038,0,500
3,PESHI,5LwseQRo8fsz4S3y7jbqqe5C7tZTz5PwhXNCHj13jLBi,2025-02-26 18:09:56,457912,206599,10,False,61285,1,400
4,Frog & Toad,EixRoyVqcVRZHv56GVhnUUjzF23shGsXJVKp2rPspump,2025-02-26 18:10:51,7227,76857,50,True,29137,0,500
...,...,...,...,...,...,...,...,...,...,...
766,Burger King,BGsy5ySAPYTHjFjmWWrzmU4XtXoNq1ZynM352p8Epump,2025-03-08 15:17:05,1107,152265,10,True,37344,0,501
767,Literally a slow cook,AuAzBe6C3rgcb4qNuCjP5RpEGaobwnx5VP75MdeSpump,2025-03-08 15:19:06,760,101589,10,True,33849,0,120501
768,Leonardo JD Vance,ErkJdxQDjRTYwE6e8ajzTwMxzj7VULN5SQyHMW84pump,2025-03-08 15:58:10,6,11953905,10,True,321772,0,10501
769,Anxiety,454Qsd6WX3azk4xTYtg1jre3fEBJbJRMc3AdMk3Xpump,2025-03-08 20:08:08,3419,84405,10,True,30574,0,2401


In [4]:
def parse_price_history(df, tp=35, sl=-40):
    """
    Expands the PriceHistory column into multiple rows while retaining original columns.
    Calculates price variation percentage and time since the boost.
    """
    df = df.copy()

    # Limpiar y formatear PriceHistory antes de parsearlo
    df['PriceHistory'] = df['PriceHistory'].astype(str).str.replace(r'\\"', '"', regex=True).str.strip('"')
    df = df[df['PriceHistory'].notna() & (df['PriceHistory'] != 'nan') & (df['PriceHistory'] != '')]

    # Parsear PriceHistory
    df['PriceHistory'] = df['PriceHistory'].apply(json.loads)
    df = df[df['PriceHistory'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

    # Extraer StartPrice con vectorización
    df['StartPrice'] = [float(x[0].get('price', x[0].get('Price', 0))) if isinstance(x, list) and x else None for x in df['PriceHistory']]
    
    # Expandir PriceHistory a múltiples filas
    df_expanded = df.explode('PriceHistory', ignore_index=True)
    
    if df_expanded.empty:
        return pd.DataFrame(columns=['TokenMint', 'TokenName', 'DetectedAt', 'PriceTime', 'price', 'PriceVariation_%', 'Trigger', 'TimeSinceBoostStart'])
    
    # Normalizar datos de precios
    price_data = pd.json_normalize(df_expanded['PriceHistory'])
    
    # Determinar nombres de columnas dinámicamente
    price_data.rename(columns={
        'time': 'PriceTime', 'Time': 'PriceTime',
        'price': 'price', 'Price': 'price'
    }, inplace=True)
    
    if 'PriceTime' not in price_data.columns or 'price' not in price_data.columns:
        return pd.DataFrame(columns=['TokenMint', 'TokenName', 'DetectedAt', 'PriceTime', 'price', 'PriceVariation_%', 'Trigger', 'TimeSinceBoostStart'])
    
    # Convertir tipos de datos
    price_data['PriceTime'] = pd.to_datetime(price_data['PriceTime'], errors='coerce').dt.tz_localize(None)
    price_data['price'] = pd.to_numeric(price_data['price'], errors='coerce')
    
    # Fusionar con el DataFrame original
    df_expanded = df_expanded.drop(columns=['PriceHistory']).reset_index(drop=True)
    df_expanded = pd.concat([df_expanded, price_data], axis=1)
    
    # Calcular el primer PriceTime por TokenMint
    first_price_time = df_expanded.groupby('TokenMint')['PriceTime'].transform('min')
    
    # Filtrar tokens cuyo primer PriceTime sea más de 1 minuto después de DetectedAt
    mask_valid_tokens = first_price_time <= df_expanded['DetectedAt'] + pd.Timedelta(minutes=1)
    df_expanded = df_expanded[mask_valid_tokens]
    
    # Calcular TimeSinceBoostStart (en segundos) con vectorización
    df_expanded['TimeSinceBoostStart'] = (df_expanded['PriceTime'] - df_expanded['DetectedAt']).dt.total_seconds().astype('int32').clip(lower=0)
    
    # Calcular variación porcentual del precio
    df_expanded['PriceVariation_%'] = ((df_expanded['price'] - df_expanded['StartPrice']) / df_expanded['StartPrice'] * 100).round(2)
    
    # Determinar triggers con np.select (vectorización)
    conditions = [
        df_expanded['PriceVariation_%'] >= tp,
        df_expanded['PriceVariation_%'] <= sl
    ]
    choices = ['TP', 'SL']
    df_expanded['Trigger'] = np.select(conditions, choices, default='No event')
    
    # Definir tipos de datos
    dtypes = {
        'price': 'float32',
        'TimeSinceBoostStart': 'int32',
        'Trigger': 'str'
    }
    
    return df_expanded.astype(dtypes).reset_index(drop=True)



price_data = parse_price_history(processed_data)
price_data


,DetectedAt,TokenAge,PubKey,IsLP,Amount,TokenName,TokenMint,IsPump,MarketCap,TotalLiquidity,TotalLPProviders,RugScore,StartPrice,price,PriceTime,TimeSinceBoostStart,PriceVariation_%,Trigger
0,2025-02-26 18:00:03,8,6ovgspym4xo1FcaFhquhb4AzrMbQkGFuLhmx2ZeLPjJ3,False,10,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,False,1416408467,341774,0,10500,0.000143,0.000143,2025-02-26 18:00:13.540,10,0.00,No event
1,2025-02-26 18:00:03,8,6ovgspym4xo1FcaFhquhb4AzrMbQkGFuLhmx2ZeLPjJ3,False,10,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,False,1416408467,341774,0,10500,0.000143,0.000143,2025-02-26 18:00:23.850,20,0.00,No event
2,2025-02-26 18:00:03,8,6ovgspym4xo1FcaFhquhb4AzrMbQkGFuLhmx2ZeLPjJ3,False,10,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,False,1416408467,341774,0,10500,0.000143,0.000143,2025-02-26 18:00:34.340,31,0.19,No event
3,2025-02-26 18:00:03,8,6ovgspym4xo1FcaFhquhb4AzrMbQkGFuLhmx2ZeLPjJ3,False,10,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,False,1416408467,341774,0,10500,0.000143,0.000143,2025-02-26 18:00:44.736,41,0.55,No event
4,2025-02-26 18:00:03,8,6ovgspym4xo1FcaFhquhb4AzrMbQkGFuLhmx2ZeLPjJ3,False,10,kanye coin,6rXcf4Kpdf2WYTgp5ozPK4ZSGcDoiJnXp6vsnjrW5e1P,False,1416408467,341774,0,10500,0.000143,0.000144,2025-02-26 18:00:55.066,52,0.71,No event
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1450822,2025-03-08 20:09:08,194,88qidXxscsjGpWRR8a9sxHezDZvf6NmcmX4tTWojwK6u,False,500,COCORO,6qRtLxaBCJgnuujZ2FzN2hinkhAMQHagRCRwMestpump,True,4705067,229242,0,501,0.004786,0.005100,2025-03-08 20:12:43.763,215,6.56,No event
1450823,2025-03-08 20:09:08,194,88qidXxscsjGpWRR8a9sxHezDZvf6NmcmX4tTWojwK6u,False,500,COCORO,6qRtLxaBCJgnuujZ2FzN2hinkhAMQHagRCRwMestpump,True,4705067,229242,0,501,0.004786,0.005111,2025-03-08 20:12:54.095,226,6.80,No event
1450824,2025-03-08 20:09:08,194,88qidXxscsjGpWRR8a9sxHezDZvf6NmcmX4tTWojwK6u,False,500,COCORO,6qRtLxaBCJgnuujZ2FzN2hinkhAMQHagRCRwMestpump,True,4705067,229242,0,501,0.004786,0.005066,2025-03-08 20:13:04.475,236,5.86,No event
1450825,2025-03-08 20:09:08,194,88qidXxscsjGpWRR8a9sxHezDZvf6NmcmX4tTWojwK6u,False,500,COCORO,6qRtLxaBCJgnuujZ2FzN2hinkhAMQHagRCRwMestpump,True,4705067,229242,0,501,0.004786,0.005067,2025-03-08 20:13:14.858,246,5.88,No event


In [5]:
def token_behavior_summary(df, max_seconds=3000, window=2, threshold=-60):
    """
    Returns summary of token behavior in max_seconds timestamp.
    Args:
        - df (pd.DataFrame): Data to extract metrics for summary
        - max_seconds (int): Maximum seconds of behavior analysis for each token.
    
    Returns:
        pd.DataFrame: Dataframe with summary metrics, such as max_price_variation and its seconds since the boost started, etc...
    """
    df = df.copy()

    # Filter all transactions with less than max_seconds parameters
    df = df[df['TimeSinceBoostStart'] <= max_seconds]

    # Group data for token so we can extract metrics for each individual token
    df_grouped = df.groupby('TokenMint')

    summary_list = []

    for tokenmint, group in df_grouped:
        group = group.sort_values(by='TimeSinceBoostStart')

        first_tp_sl = group[group['Trigger'].isin(['TP', 'SL'])]
        if not first_tp_sl.empty:
            first_event = first_tp_sl.iloc[0]  # First TP or SL found
        else:
            first_event = group.iloc[0]  # If there are no TP or SL, take 1st row entrance (No Trigger)
        
        # Get last event for No Trigger rows
        last_event = group.iloc[-1]

        # Handle NaN values when finding max/min price variations
        valid_variations = group['PriceVariation_%'].dropna()
        if not valid_variations.empty:
            max_price_row = group.loc[valid_variations.idxmax()]
            min_price_row = group.loc[valid_variations.idxmin()]
        else:
            max_price_row = group.iloc[0]
            min_price_row = group.iloc[0]

        # Create binary HasRugPull column based on PriceVariation_% threshold
        price_variation_change = group['PriceVariation_%'].diff(periods=window)
        has_rugpull = (price_variation_change <= threshold).any()
        rugpull_flag = 1 if has_rugpull else 0
        
        # Determine when the RugPull happened
        rugpull_event = group.loc[price_variation_change <= threshold, 'TimeSinceBoostStart']
        rugpull_seconds = rugpull_event.min() if not rugpull_event.empty else np.nan
        
        # Start building summary list
        summary_list.append({
            'TokenMint': tokenmint, 
            'FirstTrigger': first_event['Trigger'],
            'SecondsTrigger': first_event['TimeSinceBoostStart'],

            'MaxPriceVar': max_price_row['PriceVariation_%'],
            'MaxPriceSeconds': max_price_row['TimeSinceBoostStart'],

            'MinPriceVar': min_price_row['PriceVariation_%'],
            'MinPriceSeconds': min_price_row['TimeSinceBoostStart'],

            f'LastVariation_{max_seconds}(s)': last_event['PriceVariation_%'],

            'HasRugPull': rugpull_flag,
            'RugPullSeconds': rugpull_seconds
        })

    # Convert list into DataFrame
    summary_df = pd.DataFrame(summary_list)

    return summary_df


summary_metrics = token_behavior_summary(price_data)


In [6]:
summary_metrics

,TokenMint,FirstTrigger,SecondsTrigger,MaxPriceVar,MaxPriceSeconds,MinPriceVar,MinPriceSeconds,LastVariation_3000(s),HasRugPull,RugPullSeconds
0,14b14Ppqe5fpnSJZiJviU6Lhx6XarZkxPcGNJvpApump,SL,1191,11.43,329,-49.92,2753,-49.92,0,NaN
1,1xdtu7y3LkkrVCAbm5KGKfYzq1qgKhxxk5AaJBqpump,SL,901,3.07,96,-58.52,1529,-45.33,0,NaN
2,22amBVCHPF1JgF181jxrNe5nDwg6Ta3NEh8n2QvRpump,No event,3,8.44,848,-0.00,3,0.94,0,NaN
3,22ipKuscKiufBCGaqQSqUMDdW2YUCGVytNfkYmS8pump,TP,299,47.82,506,-86.24,2989,-84.72,0,NaN
4,25WQ47bv2CvTU5MmvXMMueJzVRc46GSmpAySjwuepump,No event,2,5.10,445,-8.73,2523,-8.73,0,NaN
...,...,...,...,...,...,...,...,...,...,...
726,yEUKKJVAv2EXQyFoiDVYRn3iyaNKzAcH3irTJigpump,SL,241,5.05,20,-74.63,2669,-73.83,0,NaN
727,ymndetqtokZgHdTGnwShEJYBbTFSCfS5GcpvPo7pump,TP,537,74.78,840,-93.99,2876,-92.50,0,NaN
728,z88ngX2UhZ7tpoCTbSohPdPi1Z7d77n7mkdF3Mvpump,SL,467,-0.00,9,-66.87,2362,-66.87,0,NaN
729,zMxMfhEPmVaTSVUCnEoRQL2T3qBFe3LHYXt1zGQpump,TP,324,148.02,1348,-92.19,1618,-91.63,1,1598.0


In [46]:
summary_metrics['FirstTrigger'].value_counts()

FirstTrigger
TP          303
No event    240
SL          188
Name: count, dtype: int64

In [14]:
final_data = data_to_merge.merge(summary_metrics, on='TokenMint', how='right')
final_data

,TokenName,TokenMint,DetectedAt,TokenAge,MarketCap,Amount,IsPump,TotalLiquidity,TotalLPProviders,RugScore,FirstTrigger,SecondsTrigger,MaxPriceVar,MaxPriceSeconds,MinPriceVar,MinPriceSeconds,LastVariation_3000(s),HasRugPull,RugPullSeconds
0,JUSTICE FOR LAKEN RILEY,14b14Ppqe5fpnSJZiJviU6Lhx6XarZkxPcGNJvpApump,2025-03-05 04:29:42,150282,28750,10,True,20219,0,501,SL,1191,11.43,329,-49.92,2753,-49.92,0,NaN
1,MANSORY,1xdtu7y3LkkrVCAbm5KGKfYzq1qgKhxxk5AaJBqpump,2025-02-27 01:52:28,202,16852948,500,True,397092,0,10501,SL,901,3.07,96,-58.52,1529,-45.33,0,NaN
2,MonkanA,22amBVCHPF1JgF181jxrNe5nDwg6Ta3NEh8n2QvRpump,2025-02-27 12:47:05,742,42176,10,True,19909,0,501,No event,3,8.44,848,-0.00,3,0.94,0,NaN
3,Viral Angry Squirrel,22ipKuscKiufBCGaqQSqUMDdW2YUCGVytNfkYmS8pump,2025-03-08 07:26:09,6,77839,10,True,27703,0,10101,TP,299,47.82,506,-86.24,2989,-84.72,0,NaN
4,RETARD RETIREMENT ACCOUNT,25WQ47bv2CvTU5MmvXMMueJzVRc46GSmpAySjwuepump,2025-03-01 12:53:24,13145,60744,100,True,29528,0,252501,No event,2,5.10,445,-8.73,2523,-8.73,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,can a trencher go to millions?,yEUKKJVAv2EXQyFoiDVYRn3iyaNKzAcH3irTJigpump,2025-03-01 05:23:24,52,48722,10,True,22559,0,254901,SL,241,5.05,20,-74.63,2669,-73.83,0,NaN
727,Reversed GrokCoin,ymndetqtokZgHdTGnwShEJYBbTFSCfS5GcpvPo7pump,2025-03-07 13:30:44,5,60800,50,True,24786,0,501,TP,537,74.78,840,-93.99,2876,-92.50,0,NaN
728,please market do something,z88ngX2UhZ7tpoCTbSohPdPi1Z7d77n7mkdF3Mvpump,2025-02-28 14:17:04,1,55006,10,True,22532,1,2801,SL,467,-0.00,9,-66.87,2362,-66.87,0,NaN
729,Digital Gold,zMxMfhEPmVaTSVUCnEoRQL2T3qBFe3LHYXt1zGQpump,2025-03-03 14:55:38,3,75403,500,True,29014,0,501,TP,324,148.02,1348,-92.19,1618,-91.63,1,1598.0


In [15]:
# Convert final_data to csv
final_data.to_csv('data/processed_data.csv', index=False)